## Antiguo proyecto de Redes Neuronales Recurrentes usando audios .wav, obteniendo su MFCC, aplicandolos a una RNN y validandolo con k fold cross validation

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
import os
import soundfile as sf
import json
import random
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from keras import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
import datetime
from sklearn.metrics import confusion_matrix
random.seed(0)

#### Conseguir los paths

La función consigue los caminos a los archivos de audio .wav pasándole una dirección del directorio y devolviendo el path de cada audio del dataset.

In [2]:
def get_files_from_path(directory):
    path_files = []
    dir_list = os.listdir(directory)
    for path in dir_list:
        path_files.append(directory+"\\"+path)
    return path_files

Con la lista de paths a los audios se va a recoger las emociones que necesitemos buscándolo por el código de emoción en el nombre del archivo para su distinción.

(Es necesario conocer en qué posición está el código en el path, varía según el nivel que se encuentre en el path completo).


In [40]:
def extract_paths_for_emotions_keys(emotions_code, files_path, get_code):
    paths = []
    emotions_set = set(emotions_code)
    for code_file in files_path:
        if (get_code(code_file) in emotions_set):
            paths.append(code_file)
    return paths

def get_code_crema_d(path):
    return path[107:110]

def get_code_savee(path):
    return path[96]

Esta función abre el archivo .wav y obtiene el mfcc escalado en un vector de 40 elementos.

In [16]:

audio, sample_rate = librosa.load("C:\\Users\\bacs2\\Downloads\\Taller De Grado\\Previous\\RNNC\\Datasets\\ALL\\DC_a07.wav", res_type='kaiser_fast')
mfccs_features = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
print(f"{len(mfccs_features)} {len(mfccs_features[0])}")
for j in mfccs_features:
    for i in j:
        print(f"{i:.2f}",end=' ')
    print("\n")
mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
print(len(mfccs_scaled_features))
for i in mfccs_scaled_features:
    print(f"{i:.2f}",end=' ')

40 116
-363.19 -414.01 -476.62 -475.43 -476.37 -476.93 -476.12 -475.94 -478.01 -478.31 -476.99 -477.34 -476.64 -476.62 -477.80 -477.81 -478.84 -478.01 -478.52 -476.89 -476.33 -466.54 -435.83 -412.28 -396.43 -366.37 -336.05 -312.91 -292.35 -267.67 -273.43 -277.80 -222.03 -198.07 -199.80 -198.53 -210.20 -215.38 -237.43 -263.72 -211.96 -148.65 -111.44 -102.54 -92.93 -96.31 -117.98 -141.85 -159.07 -183.06 -182.91 -177.43 -167.90 -163.05 -192.20 -234.59 -262.49 -280.98 -226.94 -152.27 -112.53 -104.43 -91.73 -74.93 -87.29 -111.80 -129.17 -147.03 -197.58 -238.17 -217.70 -188.33 -152.94 -90.41 -47.05 -53.63 -85.62 -83.11 -76.11 -76.32 -91.48 -133.28 -176.23 -211.73 -237.58 -218.68 -238.41 -281.33 -323.38 -373.03 -423.87 -464.05 -484.20 -489.87 -486.68 -486.84 -487.36 -488.04 -486.87 -490.70 -493.23 -491.08 -491.00 -489.17 -485.12 -486.27 -489.61 -489.90 -488.60 -480.82 -480.20 -483.12 -469.09 -457.49 -463.71 -440.86 

110.95 93.13 38.92 40.37 39.02 38.27 39.44 39.77 37.05 36.72 38.43 38.03 39.

In [2]:
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features

Esta función permite guardar los MFCC en 'features', el código en 'code' y su dirección en 'path' en un archivo json. 

In [5]:
def save_elements_in_json(examples_saved, name):
    json_files = []
    json_file = {}
    index = 0
    for file in examples_saved:
        json_file = {"id": index, "features":[str(elem) for elem in file[0]] ,"code":file[1], "path":file[2]}
        json_files.append(json_file)
        index += 1
    json_object = json.dumps(json_files)
    with open(f"{name}.json", "w") as outfile:
        outfile.write(json_object)

La función permite cargar datos del MFCC y código desde un archivo json.

In [6]:
def load_elements_from_json(name):
    f = open(f'{name}.json')
    data = json.load(f)
    examples = []
    for element in data:
        examples.append(([float(feature) for feature in (element["features"])], element["code"]))
    return examples

La función nos permite devolver una lista de MFCC obtenidos de una lista de paths. 


El MFCC tiene un límite que no le permite cargar archivos menor o igual a 44 kb.

Con el diccionario obtenemos el total de audios recuperados por emoción.


In [37]:
def get_features(paths,get_code, files_filters = dict()):
    examples = []
    for path in paths:
        code = get_code(path)
        file_stats = os.stat(path)
        if (file_stats.st_size > 44):
            feature = features_extractor(path)
            files_filters[code]+= 1
            examples.append((feature,code))
    print(f"Se obtuvo el MFCC de unos {len(paths)} sobre {sum(files_filters[files] for files in files_filters)} audios.")
    return examples

Selecciona n ejemplos que necesitemos y los mezcla.

In [92]:
def select_elements(examples, code, quantity, new_code):
    random.shuffle(examples)
    elements = []
    counter = 1
    for example in examples:
        if (counter > quantity):
            break
        if code == example[1]:
            elements.append((example[0],new_code))
            counter = counter + 1
    return elements


Los siguientes bloques obtienen los paths y filtra las emociones que necesitemos en cada dataset (CREMA-D y SAVEE).

In [33]:
files_path = get_files_from_path(f"{os.getcwd()}\\..\\Datasets\\AudioWav")
emotions_code = ["NEU", "FEA","ANG"]
datas_files = extract_paths_for_emotions_keys(emotions_code, files_path, get_code_crema_d)

In [34]:
files_path_s = get_files_from_path(f"{os.getcwd()}\\..\\Datasets\\ALL")
emotions_code_s = ["a", "f","n"]
datas_files_s = extract_paths_for_emotions_keys(emotions_code_s, files_path_s, get_code_savee) 

In [35]:
examples = []
files_filters = dict()
files_filters["NEU"] = 0
files_filters["FEA"] = 0
files_filters["ANG"] = 0
files_filters["a"] = 0
files_filters["f"] = 0
files_filters["n"] = 0

Obtenemos los MFCC con su código de emoción para posteriormente unir los datos de ambos datasets.

In [36]:
examples = get_features(datas_files, get_code_crema_d, files_filters)
examples_s = get_features(datas_files_s, get_code_savee, files_filters)
print(f"transcurrio {round((end.microseconds/1000000),2)} s")
es = examples + examples_s
entries = []
for example in es:
    entries.append((example[0], example[1]))

Se obtuvo el MFCC de unos 3629 sobre 6 audios.
Se obtuvo el MFCC de unos 240 sobre 6 audios.
transcurrio 0.27 s


Obtendremos una cantidad de datos y cambiamos las claves a los labels {"without_stress" y "stress"} posteriormente se intercambia los datos, guardamos en X los MFCC e en y los labels, estos también se categorizan.

In [93]:
entries = []
for example in es:
    entries.append((example[0], example[1]))
datas = select_elements(entries, 'NEU', 896,"without_stress")
datas += select_elements(entries, 'ANG', 550, "stress")
datas += select_elements(entries, 'FEA', 550, "stress")
datas += select_elements(entries, 'a', 60, "stress")
datas += select_elements(entries, 'f', 60, "stress")
datas += select_elements(entries, 'n', 120, "without_stress")
random.shuffle(datas)

In [96]:
X = []
y = []
for data in datas:
    X.append(data[0])
    y.append(data[1])

In [99]:
labelencoder=preprocessing.LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))
y = np.array(y)

Se separa los datos en una parte para el entrenamiento y en otro para el testeo a partir de un porcentaje (0.8, 0.2).

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2,random_state=0)

In [101]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

1788 448 1788 448


In [102]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

Esta es la implementación y arquitectura de la red neuronal.

In [103]:
#creacion del modelo
num_labels = y.shape[1]
dim_entrada = (X_train.shape[1],1)
    
#definiendo modelo
model = Sequential()
model.add(LSTM(units=50,input_shape= dim_entrada))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

Aqui se entrena el modelo y realizará varias iteraciones, posteriormente se puede conseguir el val_loss y el val_accuracy.

In [104]:
#numero de epocas
num_epochs = 50
num_batch_size = 32
start = datetime.datetime.now()
   
model.fit(X_train, y_train, batch_size=num_batch_size,epochs=num_epochs, validation_data=(X_test, y_test))
duration = datetime.datetime.now() - start

Epoch 1/50
56/56 [==============================] - 7s 61ms/step - loss: 0.6754 - accuracy: 0.5744 - val_loss: 0.6507 - val_accuracy: 0.6094
Epoch 2/50
56/56 [==============================] - 2s 39ms/step - loss: 0.6300 - accuracy: 0.6566 - val_loss: 0.6553 - val_accuracy: 0.6272
Epoch 3/50
56/56 [==============================] - 2s 34ms/step - loss: 0.6008 - accuracy: 0.6706 - val_loss: 0.6198 - val_accuracy: 0.6384
Epoch 4/50
56/56 [==============================] - 2s 41ms/step - loss: 0.5687 - accuracy: 0.7075 - val_loss: 0.6046 - val_accuracy: 0.6384
Epoch 5/50
56/56 [==============================] - 2s 38ms/step - loss: 0.5618 - accuracy: 0.7109 - val_loss: 0.5807 - val_accuracy: 0.6585
Epoch 6/50
56/56 [==============================] - 3s 49ms/step - loss: 0.5407 - accuracy: 0.7092 - val_loss: 0.5819 - val_accuracy: 0.6607
Epoch 7/50
56/56 [==============================] - 3s 45ms/step - loss: 0.5339 - accuracy: 0.7265 - val_loss: 0.5505 - val_accuracy: 0.6786
Epoch 8/50
56

In [105]:
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"val_loss: {test_accuracy[0]}", f"val_accuracy: {test_accuracy[1]}")

val_loss: 0.5421816110610962 val_accuracy: 0.7254464030265808


In [106]:
y_values = model.predict(X_test)
y_prediction=[([1,0] if i[0]>i[1] else [0,1]) for i in y_values]

14/14 [==============================] - 1s 22ms/step


In [107]:
y_i = len(y_values)
i = 0
true_values = 0
while (i < y_i):
    true_values += (1 if (y_test[i][0] == y_prediction[i][0] or y_test[i][1] == y_prediction[i][1]) else 0)
    i = i + 1 

In [108]:
print(f"El algoritmo acerto {true_values} veces sobre los {y_i} casos.")

El algoritmo acerto 325 veces sobre los 448 casos.


El siguiente archivo guarda la configuración y los pesos.

In [28]:
# serialize model to JSON
model_json = model.to_json()
with open("model_5.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_5.h5")
print("Saved model to disk")

Saved model to disk


Esta línea guarda los datos obtenidos en cada corrida.

In [29]:
#0.8125
'''
accuracy 
val_loss: 0.4896196722984314 val_accuracy: 0.7209821343421936
val_loss: 0.5390793085098267 val_accuracy: 0.7388392686843872
val_loss: 0.3872809410095215 val_accuracy: 0.8102678656578064 model_2 
val_loss: 0.41717106103897095 val_accuracy: 0.8035714030265808 model_3
val_loss: 0.4412716329097748 val_accuracy: 0.7767857313156128
val_loss: 0.43090111017227173 val_accuracy: 0.7901785969734192
val_loss: 0.4307461678981781 val_accuracy: 0.8013392686843872
val_loss: 0.44158974289894104 val_accuracy: 0.7410714030265808
val_loss: 0.5022664070129395 val_accuracy: 0.7455357313156128
val_loss: 0.4658648669719696 val_accuracy: 0.7700892686843872
val_loss: 0.46482276916503906 val_accuracy: 0.7633928656578064
val_loss: 0.46192440390586853 val_accuracy: 0.765625
val_loss: 0.4675053656101227 val_accuracy: 0.7388392686843872
val_loss: 0.42560023069381714 val_accuracy: 0.7924107313156128
val_loss: 0.44755128026008606 val_accuracy: 0.7700892686843872
val_loss: 0.4769285023212433 val_accuracy: 0.7522321343421936
val_loss: 0.4537601172924042 val_accuracy: 0.7544642686843872
val_loss: 0.4555659890174866 val_accuracy: 0.7611607313156128
val_loss: 0.459335058927536 val_accuracy: 0.75                model_4
val_loss: 0.4552021026611328 val_accuracy: 0.7723214030265808
val_loss: 0.46012476086616516 val_accuracy: 0.7321428656578064
val_loss: 0.4430236220359802 val_accuracy: 0.7723214030265808
val_loss: 0.46520107984542847 val_accuracy: 0.7678571343421936
val_loss: 0.4261465072631836 val_accuracy: 0.7946428656578064
val_loss: 0.4542466700077057 val_accuracy: 0.7700892686843872
val_loss: 0.4542466700077057 val_accuracy: 0.7700892686843872
val_loss: 0.4661361873149872 val_accuracy: 0.7388392686843872
val_loss: 0.4661361873149872 val_accuracy: 0.7388392686843872
val_loss: 0.5038288831710815 val_accuracy: 0.7477678656578064
val_loss: 0.4211486876010895 val_accuracy: 0.7879464030265808
val_loss: 0.4537201523780823 val_accuracy: 0.7433035969734192
val_loss: 0.41213124990463257 val_accuracy: 0.765625
val_loss: 0.44190025329589844 val_accuracy: 0.7834821343421936
val_loss: 0.4493030905723572 val_accuracy: 0.7879464030265808
val_loss: 0.6083762049674988 val_accuracy: 0.7008928656578064
val_loss: 0.4399203956127167 val_accuracy: 0.7901785969734192
val_loss: 0.4300425052642822 val_accuracy: 0.7991071343421936
val_loss: 0.43003806471824646 val_accuracy: 0.7767857313156128
val_loss: 0.4450111389160156 val_accuracy: 0.7611607313156128
val_loss: 0.4859310984611511 val_accuracy: 0.7209821343421936
val_loss: 0.4417710602283478 val_accuracy: 0.7700892686843872
val_loss: 0.4944555461406708 val_accuracy: 0.7321428656578064
val_loss: 0.5189602971076965 val_accuracy: 0.7276785969734192
val_loss: 0.4687376022338867 val_accuracy: 0.7834821343421936
val_loss: 0.45363569259643555 val_accuracy: 0.7790178656578064
val_loss: 0.5052199959754944 val_accuracy: 0.7254464030265808
val_loss: 0.42957931756973267 val_accuracy: 0.7857142686843872
val_loss: 0.43377751111984253 val_accuracy: 0.7678571343421936
val_loss: 0.5221102833747864 val_accuracy: 0.7410714030265808
val_loss: 0.40920349955558777 val_accuracy: 0.7946428656578064
val_loss: 0.43967631459236145 val_accuracy: 0.7991071343421936
val_loss: 0.493486613035202 val_accuracy: 0.7232142686843872
val_loss: 0.44643810391426086 val_accuracy: 0.7700892686843872
val_loss: 0.43446046113967896 val_accuracy: 0.7790178656578064
val_loss: 0.4538533687591553 val_accuracy: 0.7767857313156128
val_loss: 0.5259029269218445 val_accuracy: 0.75
val_loss: 0.4805241525173187 val_accuracy: 0.7544642686843872
val_loss: 0.4938252866268158 val_accuracy: 0.7165178656578064
val_loss: 0.3968351483345032 val_accuracy: 0.7879464030265808
val_loss: 0.46318870782852173 val_accuracy: 0.7455357313156128
val_loss: 0.4486755430698395 val_accuracy: 0.7745535969734192
val_loss: 0.4509487748146057 val_accuracy: 0.7700892686843872
val_loss: 0.45683008432388306 val_accuracy: 0.7366071343421936
val_loss: 0.41022810339927673 val_accuracy: 0.7700892686843872
val_loss: 0.4046490788459778 val_accuracy: 0.7924107313156128
val_loss: 0.5198861956596375 val_accuracy: 0.7455357313156128
val_loss: 0.5119696259498596 val_accuracy: 0.7366071343421936
val_loss: 0.5051974058151245 val_accuracy: 0.765625
val_loss: 0.4657873809337616 val_accuracy: 0.7611607313156128
val_loss: 0.4428311288356781 val_accuracy: 0.7767857313156128
val_loss: 0.4056004583835602 val_accuracy: 0.7946428656578064
val_loss: 0.4947616457939148 val_accuracy: 0.7388392686843872
val_loss: 0.4718573987483978 val_accuracy: 0.7790178656578064
val_loss: 0.44807955622673035 val_accuracy: 0.7589285969734192
val_loss: 0.4626654088497162 val_accuracy: 0.7589285969734192
val_loss: 0.49828478693962097 val_accuracy: 0.7700892686843872
val_loss: 0.49322938919067383 val_accuracy: 0.7433035969734192
val_loss: 0.4238942265510559 val_accuracy: 0.7790178656578064
val_loss: 0.460832804441452 val_accuracy: 0.7455357313156128
val_loss: 0.46732380986213684 val_accuracy: 0.7589285969734192
val_loss: 0.40039685368537903 val_accuracy: 0.7745535969734192
val_loss: 0.4678298830986023 val_accuracy: 0.7433035969734192
val_loss: 0.4330616593360901 val_accuracy: 0.7767857313156128
val_loss: 0.469801664352417 val_accuracy: 0.78125
val_loss: 0.4441928267478943 val_accuracy: 0.7700892686843872
val_loss: 0.5477870106697083 val_accuracy: 0.7388392686843872
val_loss: 0.5036300420761108 val_accuracy: 0.7410714030265808
val_loss: 0.41244035959243774 val_accuracy: 0.7946428656578064
val_loss: 0.4877232015132904 val_accuracy: 0.7366071343421936
'''

'\naccuracy \nval_loss: 0.4896196722984314 val_accuracy: 0.7209821343421936\nval_loss: 0.5390793085098267 val_accuracy: 0.7388392686843872\nval_loss: 0.3872809410095215 val_accuracy: 0.8102678656578064 model_2 \nval_loss: 0.41717106103897095 val_accuracy: 0.8035714030265808 model_3\nval_loss: 0.4412716329097748 val_accuracy: 0.7767857313156128\nval_loss: 0.43090111017227173 val_accuracy: 0.7901785969734192\nval_loss: 0.4307461678981781 val_accuracy: 0.8013392686843872\nval_loss: 0.44158974289894104 val_accuracy: 0.7410714030265808\nval_loss: 0.5022664070129395 val_accuracy: 0.7455357313156128\nval_loss: 0.4658648669719696 val_accuracy: 0.7700892686843872\nval_loss: 0.46482276916503906 val_accuracy: 0.7633928656578064\nval_loss: 0.46192440390586853 val_accuracy: 0.765625\nval_loss: 0.4675053656101227 val_accuracy: 0.7388392686843872\nval_loss: 0.42560023069381714 val_accuracy: 0.7924107313156128\nval_loss: 0.44755128026008606 val_accuracy: 0.7700892686843872\nval_loss: 0.476928502321243

Este bloque ejecuta todo el código que vimos.

In [109]:
examples = []
files_filters = dict()
files_filters["NEU"] = 0
files_filters["FEA"] = 0
files_filters["ANG"] = 0
files_filters["a"] = 0
files_filters["f"] = 0
files_filters["n"] = 0
start = datetime.datetime.now()
examples = get_features(datas_files, get_code_crema_d, files_filters)
examples_s = get_features(datas_files_s, get_code_savee, files_filters)
end = datetime.datetime.now() - start
print(f"transcurrio {round((end.microseconds/1000000),2)} s")
es = examples + examples_s
entries = []
for example in es:
    entries.append((example[0], example[1]))
datas = select_elements(entries, 'NEU', 896,"without_stress")
datas += select_elements(entries, 'ANG', 550, "stress")
datas += select_elements(entries, 'FEA', 550, "stress")
datas += select_elements(entries, 'a', 60, "stress")
datas += select_elements(entries, 'f', 60, "stress")
datas += select_elements(entries, 'n', 120, "without_stress")
random.shuffle(datas)
X = []
y = []
for data in datas:
    X.append(data[0])
    y.append(data[1])
labelencoder=preprocessing.LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2,random_state=0)
print(len(X_train),len(X_test),len(y_train),len(y_test))
y = np.array(y)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
#creacion del modelo
num_labels = y.shape[1]
dim_entrada = (X_train.shape[1],1)
    
#definiendo modelo
model = Sequential()
model.add(LSTM(units=50,input_shape= dim_entrada))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
#numero de epocas
num_epochs = 50
num_batch_size = 32
start = datetime.datetime.now()
   
model.fit(X_train, y_train, batch_size=num_batch_size,epochs=num_epochs, validation_data=(X_test, y_test))
duration = datetime.datetime.now() - start
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"val_loss: {test_accuracy[0]}", f"val_accuracy: {test_accuracy[1]}")
y_values = model.predict(X_test)
y_prediction=[([1,0] if i[0]>i[1] else [0,1]) for i in y_values]
y_i = len(y_values)
i = 0
true_values = 0
while (i < y_i):
    true_values += (1 if (y_test[i][0] == y_prediction[i][0] or y_test[i][1] == y_prediction[i][1]) else 0)
    i = i + 1 
print(f"El algoritmo acerto {true_values} veces sobre los {y_i} casos.")

Se obtuvo el MFCC de unos 3629 sobre 3628 audios.
Se obtuvo el MFCC de unos 240 sobre 3868 audios.
transcurrio 0.46 s
1788 448 1788 448
Epoch 1/50
56/56 [==============================] - 10s 76ms/step - loss: 0.6837 - accuracy: 0.5330 - val_loss: 0.6629 - val_accuracy: 0.5335
Epoch 2/50
56/56 [==============================] - 2s 41ms/step - loss: 0.6250 - accuracy: 0.6493 - val_loss: 0.5975 - val_accuracy: 0.6786
Epoch 3/50
56/56 [==============================] - 2s 41ms/step - loss: 0.6002 - accuracy: 0.6739 - val_loss: 0.5908 - val_accuracy: 0.6987
Epoch 4/50
56/56 [==============================] - 2s 36ms/step - loss: 0.5912 - accuracy: 0.6773 - val_loss: 0.5780 - val_accuracy: 0.7210
Epoch 5/50
56/56 [==============================] - 2s 41ms/step - loss: 0.5730 - accuracy: 0.6935 - val_loss: 0.5514 - val_accuracy: 0.7210
Epoch 6/50
56/56 [==============================] - 2s 35ms/step - loss: 0.5511 - accuracy: 0.7013 - val_loss: 0.5724 - val_accuracy: 0.6897
Epoch 7/50
56/56 

In [ ]:
'''
val_loss: 0.42675861716270447 val_accuracy: 0.7767857313156128
val_loss: 0.44326552748680115 val_accuracy: 0.7611607313156128
val_loss: 0.4475260078907013 val_accuracy: 0.7566964030265808
'''

#### Crossvalidation

In [110]:
from sklearn.model_selection import KFold
from sklearn import metrics

In [111]:
X = np.array(X)
y = np.array(y)
print(f"X:{X.shape} y:{y.shape}")

X:(2236, 40) y:(2236, 2)


El bloque realizara el k fold cross validation dividiendolo en 5 folds, se uso los MSE.

In [129]:
k_fold = KFold(5)
fold = 0
y_tests = []
predictions = []
results = []
for train, test in k_fold.split(X):
    fold = fold + 1
    print(f"Fold #{fold}")
    
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    print(f"Train - X:{X_train.shape} y:{y_train.shape}")
    print(f"Test - X:{X_test.shape} y:{y_test.shape}")
    
    num_labels = y.shape[1]
    dim_entrada = (X_train.shape[1],1)

    model = Sequential()
    model.add(LSTM(units=50,input_shape= dim_entrada))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    num_epochs = 50
    num_batch_size = 32

    model.fit(X_train, y_train, batch_size=num_batch_size,epochs=num_epochs, validation_data=(X_test, y_test))
    
    pred = model.predict(X_test)
    y_tests.append(y_test)
    pred=[([1,0] if i[0]>i[1] else [0,1]) for i in pred]
    predictions.append(pred)
    score = metrics.mean_squared_error(pred, y_test)
    results.append([score])
    print(f"Fold score (MSE): {score}")

y_tests = np.concatenate(y_tests)
predictions = np.concatenate(predictions)
score = metrics.mean_squared_error(pred, y_test)
print("-----------------------")
print(f"Cross-validated score (MSE): {score}")
print("-----------------------")
print("Resumen")
for result in results:
    print(f"Fold score (MSE): {result[0]}")
    
#https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_05_2_kfold.ipynb
#https://www.youtube.com/watch?v=maiQf8ray_s

Fold #1
Train - X:(1788, 40) y:(1788, 2)
Test - X:(448, 40) y:(448, 2)
Epoch 1/50
56/56 [==============================] - 7s 55ms/step - loss: 0.6818 - accuracy: 0.5330 - val_loss: 0.6681 - val_accuracy: 0.5692
Epoch 2/50
56/56 [==============================] - 2s 41ms/step - loss: 0.6364 - accuracy: 0.6443 - val_loss: 0.5988 - val_accuracy: 0.6562
Epoch 3/50
56/56 [==============================] - 2s 40ms/step - loss: 0.6073 - accuracy: 0.6695 - val_loss: 0.5759 - val_accuracy: 0.6964
Epoch 4/50
56/56 [==============================] - 1s 26ms/step - loss: 0.5801 - accuracy: 0.6952 - val_loss: 0.5607 - val_accuracy: 0.7076
Epoch 5/50
56/56 [==============================] - 2s 29ms/step - loss: 0.5774 - accuracy: 0.6840 - val_loss: 0.5574 - val_accuracy: 0.7054
Epoch 6/50
56/56 [==============================] - 2s 30ms/step - loss: 0.5535 - accuracy: 0.6980 - val_loss: 0.5501 - val_accuracy: 0.6875
Epoch 7/50
56/56 [==============================] - 2s 35ms/step - loss: 0.5515 - a

Epoch 14/50
56/56 [==============================] - 2s 36ms/step - loss: 0.4893 - accuracy: 0.7513 - val_loss: 0.5160 - val_accuracy: 0.7159
Epoch 15/50
56/56 [==============================] - 2s 35ms/step - loss: 0.5024 - accuracy: 0.7367 - val_loss: 0.4611 - val_accuracy: 0.7539
Epoch 16/50
56/56 [==============================] - 2s 32ms/step - loss: 0.4832 - accuracy: 0.7501 - val_loss: 0.4848 - val_accuracy: 0.7338
Epoch 17/50
56/56 [==============================] - 2s 38ms/step - loss: 0.4866 - accuracy: 0.7546 - val_loss: 0.4542 - val_accuracy: 0.7673
Epoch 18/50
56/56 [==============================] - 2s 38ms/step - loss: 0.4718 - accuracy: 0.7630 - val_loss: 0.4606 - val_accuracy: 0.7539
Epoch 19/50
56/56 [==============================] - 2s 29ms/step - loss: 0.4718 - accuracy: 0.7552 - val_loss: 0.4372 - val_accuracy: 0.7808
Epoch 20/50
56/56 [==============================] - 2s 27ms/step - loss: 0.4708 - accuracy: 0.7479 - val_loss: 0.4313 - val_accuracy: 0.7875
Epoch 

Epoch 27/50
56/56 [==============================] - 2s 35ms/step - loss: 0.4521 - accuracy: 0.7619 - val_loss: 0.5360 - val_accuracy: 0.7114
Epoch 28/50
56/56 [==============================] - 2s 36ms/step - loss: 0.4487 - accuracy: 0.7619 - val_loss: 0.5018 - val_accuracy: 0.7315
Epoch 29/50
56/56 [==============================] - 2s 36ms/step - loss: 0.4358 - accuracy: 0.7697 - val_loss: 0.4814 - val_accuracy: 0.7539
Epoch 30/50
56/56 [==============================] - 2s 37ms/step - loss: 0.4281 - accuracy: 0.7708 - val_loss: 0.4833 - val_accuracy: 0.7360
Epoch 31/50
56/56 [==============================] - 2s 35ms/step - loss: 0.4341 - accuracy: 0.7731 - val_loss: 0.4823 - val_accuracy: 0.7562
Epoch 32/50
56/56 [==============================] - 2s 35ms/step - loss: 0.4214 - accuracy: 0.7775 - val_loss: 0.4936 - val_accuracy: 0.7427
Epoch 33/50
56/56 [==============================] - 2s 35ms/step - loss: 0.4631 - accuracy: 0.7535 - val_loss: 0.4898 - val_accuracy: 0.7427
Epoch 

In [127]:
metrics.mean_squared_error([[1,0],[0,1],[0,1],[1,0]], [[1,0],[0,1],[0,1],[0,1]])                   

0.25

In [128]:
0.5072185855351756**2

0.25727069351230425